Chapter 4. Relabeling
=======

Recall that **Variable**s in **Prove-It** are arbitrary labels that are interchangeable. In `tutorial_01_core_expr`, we demonstrated using the `relabeled` method of **Expression** to transform an expression to one with **Variable**s swapped for other **Variable**s.  This was, however, only a transformation of the expression with no proof implication.  Here we will discuss the *relabeling* derivation step in which we can derive a new **KnownTruth** from an existing **KnownTruth** by swapping **Variable**s for other **Variable**s.

## Attempting to relabel a free Variable of an assumption

Let us take an example from the previous tutorial chapter:

In [1]:
from proveit._common_ import A, B, C
from proveit.logic import Implies
%begin relabeling

In [2]:
A_impl_B = Implies(A, B)

A_impl_B: A => B

In [3]:
B_from_A = A_impl_B.deriveConsequent(assumptions={A, A_impl_B})

B_from_A: {A => B , A} |- B

Now we will attempt to perform a relabeling derivation step by calling the `relabel` method on a **KnownTruth**.  This will not work because we are not allowed to relabel a **Variable** in the list of assumptions.  Otherwise we would end up being able to prove false statements (unless we relabeled the assumptions in a consistent manner, but **Prove-It** has a different mechanism for doing this as we shall see next).

In [4]:
from proveit import RelabelingFailure
try:
    B_from_A.relabel({B:C})
    assert False, "Expecting an RelabelingFailure error; should not make it to this point"
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Proof step failed assuming {A => B, A}: Attempting to relabel variable(s) that are free in the assumptions: {B} when relabeling B


## Basic relabeling

If we convert the assumptions to explicit hypotheses first, then we can do the relabeling that we failed to do before.

In [5]:
explicit_B_from_A = B_from_A.asImplication(A_impl_B).asImplication(A)

explicit_B_from_A: |- A => ((A => B) => B)

This is a true statement that requires no assumptions.  This statement is valid for any value of $A$ and $B$ according to the rule that an implication is true as long as the conclusion is true whenever the hypothesis is true (but otherwise being indifferent to truth-aptness).  Now we will relabel $B$ to $C$ by calling **KnownTruth**'s `relabel` method.

In [6]:
explicit_C_from_A = explicit_B_from_A.relabel({B:C})

explicit_C_from_A: |- A => ((A => C) => C)

Here is the full proof:

In [7]:
explicit_C_from_A.proof()

,step type,requirements,statement
0,relabeling,1,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAL8AAAAVBAMAAADlQyK9AAAAMFBMVEX///92dnYQEBBERERUVFS6 urqIiIhmZmaqqqrc3Nzu7u6YmJgyMjIiIiLMzMwAAACFkkdwAAAAAXRSTlMAQObYZgAAAhlJREFU OMvFlb9LI0EUx1+i67pJXMVCwcY0B3IWXnGFYqGFIIKgiGCjELDX1UZFMPkDjjOgEAvBIGgVJBZ2 glG0UBELO0FMYSvanL/hnB9vzMbZN5Y++M6PfObN7L73dgLwfWYlpMwWHTTAHTW4DaI3KIrD5u+Z fAYWAllkvHfe8ewiTs+D1pygKB5qBRhphVjgAfV5sKbzUMDXvM6WkBpWJaQoHjljo9AzROt0BrlL 1jSnYVtOV5Lp8gfnFs5KUTz2k+cpDpG4zuCAOw4BrMtpYtKXqWHsR1EUT6ZY43gA7Tqzn3ibAahI iXLJLuZ9ocXCmUIR3HkTcWJZbNR910Tst9gPHh9swJ0A9pWwR7noFEVw+1Xt+kf3TaoHqhYnDcKa 5yuA6iPR/UARvOZZzYd032uVM4snyO3pGfjlL7Gc2PcfiuC19yJETHe6739VUy5fZbFqKfjWuPvZ sgOCeVgc8PfTAejbJt6miAewRNfEfXG8kLXZgiJ45QtvPRWict8B3tkYIodtELv3hbFb9p0ogrsP vGr4Rru6ryiKJv6JslfPsb7ipbRoFftGFMX3UmDNqYWfWJR9ByM80axMl9qOIdr1ViqTDuzHpEju 9vfOisERaAxuDmfEJxYLuGvVBRhOS1H8496M00xdFcEXfR2UXWSUhUx/GQ0mzxP/7UVbzsAiBZPn MuoL6zOwyrTJ00lIma2qaIATAO99RLgmAXafIAAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
1,hypothetical reasoning,2,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAMAAAAAVBAMAAADiAfyoAAAAMFBMVEX///92dnYQEBBERERUVFS6 urqIiIhmZmaqqqrc3Nzu7u6YmJgyMjIiIiLMzMwAAACFkkdwAAAAAXRSTlMAQObYZgAAAh5JREFU OMvFlD1IHEEUx5+au8venetVEVIJIogIilWiggtZhEDgDrFU2ErLU6z9KCwFBVOkENzG7gQLy0BM 5IpcAgFNFw5FLA2bwJr4BTqz8+Zj9d5FrsmDPzPL783MzntvHsD/s4QnVN8yhcfAn7XoKYriu7eu O1yAhZo750P3VRkU/FbLqYKiOHQCpM4hW5M19wMsFiTMHPmKpOUk6QlRHOw/TCFkcnpb7RawlYEj 4bvFNUWy0qnZF6I4JC/Z0b8h3RG/tLAiW/EhJ6FX1JlKODiZQFEcWtji1h6AF/qAcTUbYDrwESb8 1R3t9AzHWRTFoc0Ba5/t0a5Zkyq6EGBjEBBuRREDmK9yO5oRLl9QFIdgdGqeH7zCP1IRq/6V+Qmn pn+AhAXYnDEKYEXEowtFcR7m9Hc2jhnsaVnnZ9tBaLtuvt9wSn+KdjhHUTwKc5FdITDLs9Sl8pO8 BgETrFr2DJ+z5xA7gOA8zLxQYgfYH32ZH15hAi6xYujQMa4OCt9uFMVt/oP5JQwR5uAQizhgN3ty IUJkMZ/sL/0TyzgOoSjOw2zdsMl7IwWO8Qw29wQs8ZBdKR9LRqMdRXEe5pMKxMp0w3gGdq8fwfW+ z5AZuVFl0iIf7aQQxVMjofsmKuOyPuAljnOsD37duQdlmlS7WhOiuCoqo1Uc14MPOm4OYo2Msiav UViJdS/SSg3Dt6h/2OuGoeUJ1bfkcX14B/jdu2D2SrlQAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
2,hypothetical reasoning,3,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAH8AAAAVBAMAAACd/CwcAAAAMFBMVEX////c3NwiIiJERESqqqqY mJh2dnZmZmYQEBDu7u4yMjJUVFTMzMyIiIi6uroAAAAs73DqAAAAAXRSTlMAQObYZgAAAd9JREFU OMtjYKAY8F0gpIL1AV7p0xDqEDa56P+7d1s9YKjBauz+37s3rwMyVkL467Ga3szAwP6PgQurHIcB A0P9AwZeiA9Y9SfAZTjhLJ6/QPybgTUBoQ2h7DxQ5/kNDBwQEZF6AbgMF1wR70eg0d8YOBsQBqyE s+4DdcQnMLyC8C7cR4QU3wYYixmok6mTgcEEYcBrOMsCiDUmMJRB9EzwD0AoyoUx+DcwMC4Guicd IccGj7TfDAxSlgwMS8GceWAfMTAUKYGAfgHMm3vvFoEM9gJx2MFySt9h4fP77nVVIN0H5j1gkC9A CmCvB3BvcmoD6RdIcizrEOEzCejZf2DTdu/eb4CkiHPFA7g37wOdcB45+mb1wcOH9yvUAD5grC5A UnMqE+5NUECjGMCzbAIsfEAxxNAC4jgAA7sBEQZKllC1QAsY9jtAvQANA2VoJJ8Huoz7CwODKZDN CFTD9QFhiTdSMmD8BWREIQXBBqRkIL8AEkOzQF76CVfDuAApGZwApRykaJRCSgY8XUDXPGJgEO5f yMBq/wseDczQRMlu/3v3LnA0r0MYYAili4H5bBUoijkEMHIJD7oAclI+gJErEwgXGWx4i4yVhA2Y hVdWmrABG/HKMhIs0ngP4JEEAKk4fDD5jM13AAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
3,modus ponens,"4, 5","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGwAAAAVBAMAAABRZRb3AAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAYJJREFU OMtjYGDgu8dAEshaBCJ5djGQCDIMgATnBwiHbQMWFXz+X1wcSxF8sCLeAiDB1QARYf+AzWQOAQaG +wFwLlgRWAdM2+s/CMUH4Kz8CUDsAOeCFSFrY7L9hNBWA2fNV2BgeL8AxoMoQtbGyfkVoS0EzpIA 4h1wyyGKkLXFMP9DaOOaAGN9YWBQl4GLQxSBdYDDhYEngO0niObdDQZ/YSH3ZebcfXBdUEV8oHDJ AIcTKwPTX6QAZIYGOifQL6fhIQJTVMvAkNIGFklxcfmIHPDHW8EUI9A9nHDXwxQx1jEwRCSAWEAr 25F0sVVBgoEfaBPfL5goTBEoXbGA/MYMCjAkv22EBl7+A6CCzzCXQxWB/cYFSi/KQFyP5DUHpGjT L4DGP0wRPAJ4QDbKI9KGOlK0sbUfYOD5zYCkCKaNSeK7AsPD/7VwbUJQ+uJ/F5dyoDt5+hWQFKGk SWSQgC7AqoDEwakNMwMxoGvjI1kbOF3x7CRC2wSU3A1OV3yLSS9LAHxvb5SGMlqOAAAAAElFTkSu QmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢"
4,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFMAAAAVBAMAAADInc9CAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAA

Note, however, that a **Variable** may only be relabeled to another **Variable**.  To do anything else, *specialization* would be required.  We will discuss *specialization* in the next tutorial chapter.

In [8]:
from proveit.logic import And
try:
    explicit_B_from_A.relabel({B:And(B, C)})
    assert False, "Expecting an RelabelingFailure error; should not make it to this point"
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Proof step failed: May only relabel a Variable to a Variable, not B to B and C when relabeling A => ((A => B) => B)


## Simultaneous relabeling

Relabeling will occur simultaneously in a consistent manner.  For example, we can switch labels.

In [9]:
explicit_B_from_A.relabel({A:B, B:A})

|- B => ((B => A) => A)

Let us try a case with three labels that we will cycle.

In [10]:
nestedImpl = Implies(A, Implies(B, C))

nestedImpl: A => (B => C)

In [11]:
CfromNestedImpl = nestedImpl.deriveConsequent([nestedImpl, A]).deriveConsequent([nestedImpl, A, B])

CfromNestedImpl: {A => (B => C) , A , B} |- C

In [12]:
cascadingImpl = CfromNestedImpl.asImplication(nestedImpl).asImplication(B).asImplication(A)

cascadingImpl: |- A => (B => ((A => (B => C)) => C))

Convincing yourself that this is correct is left as an exercise to the reader (you could make a truth table and/or check the logic that got us to this point).  In any case, now we will demonstrate cyclic relabeling.

In [13]:
relabeledCascadingImpl = cascadingImpl.relabel({A:B, B:C, C:A})

relabeledCascadingImpl: |- B => (C => ((B => (C => A)) => A))

Let us look at this proof:

In [14]:
relabeledCascadingImpl.proof()

,step type,requirements,statement
0,relabeling,1,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAUEAAAAVBAMAAAAqSEFXAAAAMFBMVEX///+qqqru7u4iIiJ2dnbc 3NyIiIjMzMxmZmYQEBC6uroyMjJERERUVFSYmJgAAADLUcuvAAAAAXRSTlMAQObYZgAAArNJREFU SMfVlk1oE0EYht9s3e7mxzT3Iu5NNBcxVyWhh6In14sIKt1TvQgNeBKRBiqoFxsLXkRoQBBFhBD0 UNH6Ax5EsLkogoT00JMi/qAiLbV+szPJ/kyzO2xOfvDOzuaZN/vtzjezC/zPcSeCZRyuRGYe+Qbw SYl/2ep03jTkEUabGu3sh2t6TYYXhRKZKc6RLgNmdUB6QX6LeuvyoKxFzZFVZK6uyrAllMhM8YyN ITUHZBjgxm/SZp9pvc40qXiAmqO2xEYdrrDZUjGzGSzTyHwFWAPi+egvOvkZvHsWC6QldnJPZmmL K2xuQcFMcWyeEtfqwF4o8Bx1Rha8NdAWR3rA5nfWOyEx3BcKmx9AwUzhzLDafUH+ggIfa0N/4st/ kh9MesZTbhnskRguCYXNOxwFMyVj7WbleZgMNQU++356jpfzXIlFmZuy9Nu8V+YBhmUhz2y6vPQH 8WZgH2bZrYwDqaYCn7GhHfKvo3F390hTW7bDa4wz3BSSzKnHiDUDDUzVeI1m6gr8NbuQf0/QltjI MWq2ggu0z7AuJJuLbuKRZhidTvc5mwDqflPgbK/4WvENvDDRu8ii+1iqEvMyDJuNh1asGRl6yM1t MhzAjb/U7xa8cii95BNFP3XZ5UxIDNeFPLOow4NWvBkER+r+WY7mbEfTN3x3sosfcjR3br1OyAxv hcLmVFvBrFMGWfbsPtNe2oznbEc73/LG6cLD1nmetrS7tszYPuAqbD4JBXORGX9Iu80gbq5sdj4u ++40J/40s599HDy9UtiG4TaXZH6FePOpxUfIr2xQarTusw1l7tV6r3MjgqVtrnBUVcz9pRt868Vx Odaivt0qXInM/hfQ5BA88CYf/PWVxOzVm9YcgkNvR8DTQonMIt4BO+0hOHA8KgOHK5FZfGNS0Z6J 5/8AW4UXwSm8NYgAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,hypothetical reasoning,2,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAUEAAAAVBAMAAAAqSEFXAAAAMFBMVEX///92dnYQEBBERERUVFS6 urqIiIhmZmaqqqrc3Nzu7u6YmJgyMjIiIiLMzMwAAACFkkdwAAAAAXRSTlMAQObYZgAAA51JREFU SMfVl09IVEEcx3+uvt19u+u6dFDwkhBBqaBIh/6BCz2SQNhFIiEKhMBuutXBImz3EHSRWijSg+Ai 1MlipbwVblZEWiTVqRAlPBZrpKkl2Pzmz755b2bfnhv4zczbz7z5+/395i3Af5yMPg94hpsX907h JMlmPBrMyPyHrsUqsae7lnU8qbJonBlWF0HPw4l168RrXc8PDw3lR+EGqQVWdDx0oeu6mUJW4h90 7eYx20dabagskmOG1TXQc/C1A6Q1y6tqAehtgQjWC7qR9+TBuJynjPPwcs6ev6j48ZCjv4mtq7Cf G0nP/4CeQ5H0UIyXoBgj9J7UqrYhHCMP06BwmPpCsr1ZyjgfS2edq8fkw4p/m8z/pwqbuQGYt7ZB z2GQ9Dobc54JNkJoNEGoiZSToHCYw156GOO8b9A+C0Ms+hxm1aSX2mYFGgVmuAa/JAKZA3QQ+2if zmlepjO4sBTAYRwgo/DAFuajjDFu5O7k7VHqeXkJs7o4mC9zCgzEmJF0NvgXtByAqGP8qCQ/FhvM HXrUxAUa8MeUm8MEXd5jxhh/QBUDMLyEaZm/s0CldLJ/mM3eASN5ZmSkJEpVx4l++y9+ZROmbGmT PZRWNEIsWHBzSIvtQkY5JGEiJe3DCDvy/VxKoc+gQF+SGdl/MDf1nGr4UVxiQRp6aku67WF6dHFY Fk6BjPKoZSXa5Wg0R0fZEFIazCuwLsmMBE3L+gVaTjXslyUAU7jqujXh9UUcfM3NYVd4NjLKDbJm OSx9b4TSDDHQSO4ooD1DkrWClqOG5ThARntBoySd023NDBmHNrqfK9IMibfUNtlSWuLiPoDv4BYk Mgr0ZZhBEN0VtByKZO9rtiQdfqITqKHxMyWfsoNDAouAdMpmxnkv3OTlMS4l5ntOWJ1nBmOuC0ni NBxOSIcT5LckelYVzuIZXhYFN2du28gZ2hT2aN8LpnilgUvp2zwoUMSBED535rQcNRxtlSLVOC9n M2BcEyNI0UbwMNn33qw9yt22txDu3Ck1rBaedJ5sdee61b0gbZGAxiI1s2MnC09232j5FfLR8U52 siNCbt1dV2kFXTeSVDisvhqiwkIWUe/1qKj4slAeHuTmxd3J/RnjvvWUz5xJJ1e/32IecNp56Wt5 pUTvkHqPBvUVOMx7MOnrqyyvlNABQoXyHJkXB7jnwcw4My9eKZ3C0JMtz5F5cTKK17+A+9y8uHfy o+4GPBoMMPsHGJc6aSJI4VgAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
2,hypothetical reasoning,3,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQEAAAAVBAMAAAC08kb3AAAAMFBMVEX///+qqqru7u4iIiJ2dnbc 3NyIiIjMzMxmZmYQEBC6uroyMjJERERUVFSYmJgAAADLUcuvAAAAAXRSTlMAQObYZgAAAt5JREFU SMfNlU9oE0EUxr+k2eZPkzQ3BRECgiK9FHMS1JYeCp6MFyn00J6iB6G9ipREilBBjBS8WNHgQepB CNWD9T+ChyLSnKRQQxb0JEpTsVKsMc6bN5vMbiabqw++3Ul/r7Mz876ZAf7neKDkx3vEVyU/Dnxv VqvvS500XGFRcxhmnqw1qp+emfo+/GHBnkBkFlKdEby4ccMqKHZHZP3uzBlIs6hZh5kj+gYoGoYf ugssC5XB6ojTNmLXbGbhX0KN9uCcRk5JxOcfMHPkp4UqLZh2kh6LVmgH+AKWh2PomHicyTLrF4nJ n+6pUSwqAdb+HZg5ZrLA1mgLrjhJBGM3gaNgeThWqZeHisVFYt9iKymmZhQrs2iM/VqRdA6sCR1v zQyP1LuYooEXROcplodHtuk5odhgBdaLdicYV0mjLBH3A39h5ICo3uRbrfzTvGh7shTCZqECy80x JYd/RLH8x9y8Lf8+n6EY4X8YsFmipxJZxcSFf3IXTvGAJMvs8o/WiANllpujaGucShk8qdv0gPR2 tMQS6w1r18ylhw5VNBaQW7Ov5RvyAsnNMZLVuCzljK1v1VXqfLDEAuaq1T8wcumhfr1EGLpNrO7s mnCd5eZoOjtDMtqImp2By2NwjUA8lmDk5CF9H4ken8hTQn5zn2EEzHFPrses4jSFWqpd5ww7K5pi IUB2h5EjL9Yusa354IT8QEKeHwW9Ci6OGr0iilMp2bsqDvIrbrNwVui1mcvjYEo78wLqFCfnhugr yTLLzeVBhjHFqZSXVtqdWCrf2UdB+r2eNnLyUHhJ28mT6r2VQuyq3ouHJ8W6LWeZR9Yb1c1X2hTj yqaxYSlrbS+Lb82nRn5dXGrPdRO/c8q9uXGF92yJ5eGYe7mQcnjHlec0bin5cW90XIOeU9nIu4bn UjHynjHePkS78q6h3c5dea8gEwXL/rx7WBWWH+8ViSzLj/vEOSU/3iPOK/nwf+a

## Duplicated relabeling

It is possible to relabel multiple **Variable**s to the same **Variable** to derive a **KnownTruth** that is weaker than the original statement.

In [15]:
redundantCascadingImpl = cascadingImpl.relabel({A:B})

redundantCascadingImpl: |- B => (B => ((B => (B => C)) => C))

In [16]:
redundantCascadingImpl.proof()

,step type,requirements,statement
0,relabeling,1,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAUQAAAAVBAMAAADMYYoTAAAAMFBMVEX///+qqqru7u4iIiJ2dnbc 3NyIiIjMzMxmZmYQEBC6uroyMjJERERUVFSYmJgAAADLUcuvAAAAAXRSTlMAQObYZgAAAxtJREFU SMfVlk9IFFEcx3+Ou87+0d29dZBgICjCS+QpqBQPYqfmFIKEnqxDoNcI2Q2LFKI1oUtFDQRhRLBY F/sjJXSIqOZSCLY4B09FqJERxmrvvd9vZmff295se+vB9zHM9/eZefPe7/fmAfzX7b7GS42gmoKp fSHp/IzDr77tlstvHTXCdFnEaqX8+WkdfILUFAz73015g5AYByG1GeeWr8ULwrsibtxmkdtqXNpi XfIlQLHOEOZJTcGxOwBzTCVAKe2EB6lpT3hp8cE/mSqBbfkXo7zLs8XMu4rZNoKSYaMR2HjMrmJb AGuAkuGuw6w7aQsv08tfx4IzP2rmR7RZ3o3ZAOu9ipm0UDKcthqA09xMzQAcBJQML/CLB+gZLA7a Wdc6GwQ+8gtCLMEbpiOWYj4kyXDKbQAu5lgXLzA4h5LgxCbvB8l7xZR1If68+qAY1WlCfD5bw6El UMxLJBmG/mg4/lusKquFWAElwcPi+w6QN8Az5tPopCee2y3aL5p2fs+sjJ49Dqq5SKrCMCn8nkIk nNjxh9RSQklw0Qv50IkZYxwLlVML7hNJh1Jtn6uaN0kKDJ1OFNy6FazsDEqCe+yQz3NSZMyYFwrs Ei/POpRqbTugmNskFTYWnAg4u+GXr7mBkuBdv/6Fl8eMCdcdmE+sYIhZt6ZgAzMYogzDhb4oOCkG tafeEBG+K2Z8vDpEk3/naq6aMUcx+5P8Vp5NUMcmKOZ1UhWmdOpeioQ7vvO+ULPQNfAqD0uEFppn DBYZJQxlT7tHO9twSTXfk2QY9kbDfLeHGB9GpoSSYL7jQx/58BUz5vx8NW7I3yIKmGrmLUs1B0gy HC81AK/nIHXZjwptOj6cYTM/Z0Ow6SQ+VMori6F5eO2X0yGAq+yQ8MyrY95DKXC73QBsrixfxK3J QUkwTLyYyvk+1DmHBEePG/B3M2mjlANOI3DQpB+gWc83ZjTHtTWNx3/umd4m4Zq/SX+EH9MdStOW xpwPHzj+FfZ3wRJK50OX7glxV2OeJjUFU+uwUTofPmofcUo3hBFUUzC1MySd3zYOfwC3LzQa79I5 2AAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
1,hypothetical reasoning,2,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAUEAAAAVBAMAAAAqSEFXAAAAMFBMVEX///92dnYQEBBERERUVFS6 urqIiIhmZmaqqqrc3Nzu7u6YmJgyMjIiIiLMzMwAAACFkkdwAAAAAXRSTlMAQObYZgAAA51JREFU SMfVl09IVEEcx3+uvt19u+u6dFDwkhBBqaBIh/6BCz2SQNhFIiEKhMBuutXBImz3EHSRWijSg+Ai 1MlipbwVblZEWiTVqRAlPBZrpKkl2Pzmz755b2bfnhv4zczbz7z5+/395i3Af5yMPg94hpsX907h JMlmPBrMyPyHrsUqsae7lnU8qbJonBlWF0HPw4l168RrXc8PDw3lR+EGqQVWdDx0oeu6mUJW4h90 7eYx20dabagskmOG1TXQc/C1A6Q1y6tqAehtgQjWC7qR9+TBuJynjPPwcs6ev6j48ZCjv4mtq7Cf G0nP/4CeQ5H0UIyXoBgj9J7UqrYhHCMP06BwmPpCsr1ZyjgfS2edq8fkw4p/m8z/pwqbuQGYt7ZB z2GQ9Dobc54JNkJoNEGoiZSToHCYw156GOO8b9A+C0Ms+hxm1aSX2mYFGgVmuAa/JAKZA3QQ+2if zmlepjO4sBTAYRwgo/DAFuajjDFu5O7k7VHqeXkJs7o4mC9zCgzEmJF0NvgXtByAqGP8qCQ/FhvM HXrUxAUa8MeUm8MEXd5jxhh/QBUDMLyEaZm/s0CldLJ/mM3eASN5ZmSkJEpVx4l++y9+ZROmbGmT PZRWNEIsWHBzSIvtQkY5JGEiJe3DCDvy/VxKoc+gQF+SGdl/MDf1nGr4UVxiQRp6aku67WF6dHFY Fk6BjPKoZSXa5Wg0R0fZEFIazCuwLsmMBE3L+gVaTjXslyUAU7jqujXh9UUcfM3NYVd4NjLKDbJm OSx9b4TSDDHQSO4ooD1DkrWClqOG5ThARntBoySd023NDBmHNrqfK9IMibfUNtlSWuLiPoDv4BYk Mgr0ZZhBEN0VtByKZO9rtiQdfqITqKHxMyWfsoNDAouAdMpmxnkv3OTlMS4l5ntOWJ1nBmOuC0ni NBxOSIcT5LckelYVzuIZXhYFN2du28gZ2hT2aN8LpnilgUvp2zwoUMSBED535rQcNRxtlSLVOC9n M2BcEyNI0UbwMNn33qw9yt22txDu3Ck1rBaedJ5sdee61b0gbZGAxiI1s2MnC09232j5FfLR8U52 siNCbt1dV2kFXTeSVDisvhqiwkIWUe/1qKj4slAeHuTmxd3J/RnjvvWUz5xJJ1e/32IecNp56Wt5 pUTvkHqPBvUVOMx7MOnrqyyvlNABQoXyHJkXB7jnwcw4My9eKZ3C0JMtz5F5cTKK17+A+9y8uHfy o+4GPBoMMPsHGJc6aSJI4VgAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
2,hypothetical reasoning,3,"⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQEAAAAVBAMAAAC08kb3AAAAMFBMVEX///+qqqru7u4iIiJ2dnbc 3NyIiIjMzMxmZmYQEBC6uroyMjJERERUVFSYmJgAAADLUcuvAAAAAXRSTlMAQObYZgAAAt5JREFU SMfNlU9oE0EUxr+k2eZPkzQ3BRECgiK9FHMS1JYeCp6MFyn00J6iB6G9ipREilBBjBS8WNHgQepB CNWD9T+ChyLSnKRQQxb0JEpTsVKsMc6bN5vMbiabqw++3Ul/r7Mz876ZAf7neKDkx3vEVyU/Dnxv VqvvS500XGFRcxhmnqw1qp+emfo+/GHBnkBkFlKdEby4ccMqKHZHZP3uzBlIs6hZh5kj+gYoGoYf ugssC5XB6ojTNmLXbGbhX0KN9uCcRk5JxOcfMHPkp4UqLZh2kh6LVmgH+AKWh2PomHicyTLrF4nJ n+6pUSwqAdb+HZg5ZrLA1mgLrjhJBGM3gaNgeThWqZeHisVFYt9iKymmZhQrs2iM/VqRdA6sCR1v zQyP1LuYooEXROcplodHtuk5odhgBdaLdicYV0mjLBH3A39h5ICo3uRbrfzTvGh7shTCZqECy80x JYd/RLH8x9y8Lf8+n6EY4X8YsFmipxJZxcSFf3IXTvGAJMvs8o/WiANllpujaGucShk8qdv0gPR2 tMQS6w1r18ylhw5VNBaQW7Ov5RvyAsnNMZLVuCzljK1v1VXqfLDEAuaq1T8wcumhfr1EGLpNrO7s mnCd5eZoOjtDMtqImp2By2NwjUA8lmDk5CF9H4ken8hTQn5zn2EEzHFPrses4jSFWqpd5ww7K5pi IUB2h5EjL9Yusa354IT8QEKeHwW9Ci6OGr0iilMp2bsqDvIrbrNwVui1mcvjYEo78wLqFCfnhugr yTLLzeVBhjHFqZSXVtqdWCrf2UdB+r2eNnLyUHhJ28mT6r2VQuyq3ouHJ8W6LWeZR9Yb1c1X2hT

## Literals cannot be relabeled

We saw that **Literals** cannot be relabeled in *expression relabeling* in the <a href="tutorial01_basic_expr.ipynb">basic expressions chapter</a>.  Let us revisit this and verify that we cannot perform a relabeling derivation step.

In [17]:
from proveit import Literal
X, Y, Z = Literal('tutorial', 'X'), Literal('tutorial', 'Y'), Literal('tutorial', 'Z')

X: tutorial
Y: tutorial
Z: tutorial

In [18]:
XYZimpl = Implies(X, Implies(Y, Z))

XYZimpl: tutorial => (tutorial => tutorial)

Let us try to relabel this.  Our first mistake will be that the expression is not a **KnownTruth**.  **Expression** has a `relabeled` method, but to call `relabel` (and make an actual derivation step), we need to start with a **KnownTruth**.

In [19]:
from proveit import RelabelingFailure
try:
    XYZimpl.relabel({Y:C})
except AttributeError as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: 'Implies' object has no attribute 'relabel'


Let us make this an axiom to make it a **KnownTruth** by fiat.  This is not normally how axioms are created, but we will get into that in a later tutorial chapter

In [20]:
from proveit import Axiom, Context
XYZimplAxiom = Axiom(XYZimpl, Context(), 'XYZimplAxiom')

XYZimplAxiom: 	step type	requirements	statement
0	axiom		|- tutorial => (tutorial => tutorial)
	tutorial.XYZimplAxiom

Now we will try to `relabel`:

In [21]:
from proveit import RelabelingFailure
try:
    XYZimplAxiom.provenTruth.relabel({Y:C})
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Proof step failed: May only relabel a Variable, not tutorial when relabeling tutorial => (tutorial => tutorial)


That time, we got the error message that only **Variable**s may be relabeled.

## Scoping violation

We also noted with *expression relabeling* in <a href="tutorial01_basic_expr.ipynb">basic expressions chapter</a> that we are not allowed to violate the scoping restrictions of **Lambda** expressions.  We revisit this for the case of the *relabeling* derivation step.

For our examples, we will invoke the infinite geometic series theorem from `proveit.number.summation`:

In [22]:
from proveit.number.summation._theorems_ import infGeomSum

In [23]:
infGeomSum

|- forall_{x in Complexes} ((Sum_{m=0}^{infinity} x^{m}) = (1 / (1 - x)))

Let us look how this expression is deconstructed in the expression information.  Note the use of **Lambda** mappings for the `Forall` *instance expression* (entry 2 below) and the `Sum` *summand* (entry 12 below).  These **Lambda** mappings define new scopes for $x$ and $m$ respectively.  This concept of operating on a **lambda** map will be discussed more in the next chapter.

In [24]:
infGeomSum.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQcAAAA/BAMAAAAf2xDgAAAAMFBMVEX///8QEBC6urrMzMxUVFRm ZmZERETc3NwyMjKqqqp2dnaIiIiYmJgiIiLu7u4AAAAU9Wx4AAAAAXRSTlMAQObYZgAABgFJREFU aN6lWU1sG0UUfuu1vbH3x0aVSgWCmBYB4kdxJdQDCDlqgQuqQg6giiDqAHJQDyQHRARVsZF6KOUn kaoKbg4SUYEIaqE2glQiFqJckHCgqCAUiCPaILUFO0nTpD/JsvbOrHftWc8bmMuuvc/jb958733v zQJwxylyVQtc0ztBbAS6kYaRHLn5gG8bfFIQxWNIu+1D9lW+GWE8KwhCL+LsZpvsjfEiKJ+Oso1T Q2Ig5FWUWYjuxh5yfWVrf/J0+WGONXbsQVmlkgTzMvmiAMFrYxBgL9m4VRDEfAFjdZBcOyiRrV0Z LNcvrDFdFgOhbmBig25afqgBYncOlATbXhoTdMUtGPqmyc1Z+s0RCL2/uXzexz7KW9nLTZ+PI1zX RVYcWKLfaOO74PyWAR9743r7+RaaQVYQpOgn13Ac6d7+tk/PPdoMAjGxsUIBY3NhPiFGxMgSd0qN BmYpiQRR6RYDYVzmThmukptBLNs74oIh+SZ/XSQ4jCvowF8SBDGd4AYH4W7gOloNrgmCSHEzy15C hZDNDd10DZ/YWhMEEePKzQjdaZsbEZMQ9a/7e02fzZ8TBKFXeSDWKdwZ+9qz7jy65yr7Jz1JMRDB ZQ4GhfIxSwiqm40I/NhHwopiINQbPPmiLMsT9shm4ych9pLzY2Igav/RVj8clM7ySmajkGgsWeu7 7yN10gaVSrcrkXrXnm0OJ8vb9c3+3q9wpUHfQ4M5bM4w7C51FfVuzd6pyoyYlhsWzT6p3bznlzAp aQap643OFYbdG19CLBm15SUWFwMBiwCv1vjn50GdljRzziZ0mawa9wnYDh32A6kqCKLTygAWKUI+ tTNIlEadDnWi5u0Mw00wCdmADXxDEMRcGbRjw8MH/J7H6KpMV/5ipESlavniu6id11YFQYxYXr6S yfwmAmLe9MTg6ydeO1kIpOEM7Dhmg1gWBWHxbRfAgK+IEhCGC0TEdOukMabNSp54CNuPQ4fqA9Fs DiZqTazhkZBLh3f3OSAI041Ft6gtunJLFNRqJMkAgR81EDqoNOsYJ58qyjvK8uG2IHT3fhgQbAot BoiYyR5VB4Ra1OnKJu/+p6ztB7i3LQhl3ZNlpSZJD18V9YQVm0buQetuYueJpFbLNtuOWH/TFoTq bc+zSW/qp5z4vT7SuO2AgZ3W1pYfl0Y7al99/YObmCwQXW4KaEPToJb/NyfgqEXuALwD8FALCFaI yh5354v9sN9b6YqCGEk4m2qlmEcymUxh6zZenvBwYGHqwoS3gPhPeQK0mjfluAzn6sJ50cUJFoiD vO51FdOCutM21aVUNBeFYO2T/MyQc1TW0IE5Z9vDvLpU2sC0oG4Bo3MvfNX3ORjP124vnH3R6QQc HRhxVPSzBgtH26f6ti2oa5hNfb3FCU5RozXiM8IGUYmLnYV4Io952kAj4fhoa3zewf5NNicGQlnj 9cO08SoRDymN+Iyu+DRt3RgQdfG1mzteR+wY0JnnqWwoR02fUCwVECBc4qvxOkyDuqpCfDxiC/Sh Hy358Slo9yYQIFziG+Imt04adzblQ4hedLDsC0J5t64nCY/4hrnlYA8JzSC+ZrsJ1/M44itxq/MS iQoNrc/KZRwIR3yz3OIrS46qlBUsCDbcZhAu8S2N8qZ0ysfbsCCCGxgQLvHt4R5kOmejjDPP6ATr RE/KYVpQl/hu4R/+0I441drwTzFf9KQKIDRkxEHUr1RPWxd4BmIMT2KOiZm+bjOmSYyqLTFqbIDO 4NRmwZJGQlShFZKnjdpxyekvprpPvUD9aIFodX1EtK7KI05pQzSV9FpAvrkBP8O3EKwl7rcDTBDc mqd5vIQ5wqDMrBRBuZiDZRhv64lUQpCXqLL4Q0ogyyXBghyHtyi8VZBaOZERdEQ4jbHSqXhbrbuU VNNKNRGubwdsgljLi7CA6LlAF+r1oTOtZZ61fKGlx5w8cVcr5KIgCOSLVHpcGc1Zf6on5X0Uu3rq uRbjpwUxhNLIXaMr34dw2x+CIB5ApjbjJ3LzN5/4BwRfECu/YC3/pGj4aWVY0BGKDfpfEDYHICuS KjwAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Literal,,
2,Lambda,parameter: 31body: 3conditions: 4,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAUwAAAA/CAMAAAC8cQG2AAAANlBMVEX///+6uroUFBQNDQ0yMjJE RETMzMwQEBAiIiKYmJju7u5mZmaqqqpUVFSIiIh2dnbc3NwAAACltDI1AAAAAXRSTlMAQObYZgAA Bf5JREFUeNrlXNmW4ygMhRmzm+3/f3awk1TboGDAqNrnjN7UXYnEBaQrWQ4hw+JcXR8SagmuoBsY knWp64NiDbLf6AYGhItQ1YdFKmTP0Q3cdynXtTf7vwTv175digH5GGAb6Baz1PVgKWWCE2U04e7W V6P7/tcveeRVnez42ej9HvS7joIWDtd5dAOdsti6Tl6XnkV90JppwvtTaIJuoC+TZ2FnLcLQCz4r 9xsVaN/XC4/sP7qBruzD6nqCd7v2Zg2CaaJ7yYi/fXKYRjYwkfhGWtX3o+mNTYBqIxbWmz11vBfU NLsA666BmcJEXZ9uoA9Ka+3VyZvu8bi30VT1GTGZ3vr8ZYa5a2CeqIwH5fqM7bK4YN41MPESxro+ g3oJXDDvGph4bFhVn8MEKS6Y7iH3nGapkCKkRh49Lpg3DcwrbbMQedZl/CJ9iMcFF8ybBubFs1jT 14TbCWwdqDIixr4gJSMymDI+Aswoq7qN+R/ssWCJXZ04e4shNIBpp1OQEQlZvsn17aJDvMMvfXAo XDDvGZhHqH1VT7E9go4afsPKdDBBA+tvt43zLS23WCU0Ia/go6CMo85b4hzjxx0Z51smhejFqKt0 Dhjwv82XbMbQbMnYGBg2v7BWRbfdkCkyKHskrxJ3GaCBITCdXJYoJBupqVkWuVkZyXVK3q1fHbTb gun2Jf74mYhdokAGBsDke8tM++hGClSZXWEJXGmaLnpz8rZue7a5cazjRwQ2dYEM9IOpiFXvGGyH nNBV/eVV4pWt/VeRDqVjGy0g9LhHyP1byMAVmOkWySR/eInS5HPAzQiYMdb1D5NvLTFCfJ9OxYg/ fgFyZoUMXIDJF5VtgCHB/gC9r8Y7G1ZnPvXeGYR1HQJzC5sVcnOwSbesKtPp5MyH41o5Npi8DmaO CwnFwxBtCD2tUidNSUXEJ7tlWc7qLAvGqn4sK7/uc2ETynOnT5vlIFN6kQxw7whm6WP5+IV7sp4o mN+54vsRGACmtuEMniA1/dD/+B42C5tta50qV2AWPuqSmCYw6QnMbcH2+Hf//iNPIk5Wm8Hcyspv F72wOQpm7JSek5n7SMUHkZ9AmPDlxRpPIw751bN06GSmzCJrCflirIL1NUZGTuZ6mYBOPnIguiya 2GLZ/L0VAJgmkDEwXXUu7mzzOmYe7wr7jWte+KiBAK/8HzC3v1NhDw3EwWDmU1fNYNJKOi5sPjBm lj5aoNxnir4Boukerqky3p4ihx/QXBHeBqgR4ZVrWtqEmAsymEsdTMBHLQGXrGUJbk3Zxps0c157 75q7e608swJUi82/zTMhH8MCtBG5SZlW2

Now we will use relabeling on the geometric series theorem to prove that we can write it in terms of $a$ and $n$ instead of $x$ and $m$.

In [25]:
from proveit._common_ import a, n, m, x
infGeomSum.relabel({x:a, m:n})

|- forall_{a in Complexes} ((Sum_{n=0}^{infinity} a^{n}) = (1 / (1 - a)))

We can also swap the labels.

In [26]:
infGeomSum.relabel({x:m, m:x})

|- forall_{m in Complexes} ((Sum_{x=0}^{infinity} m^{x}) = (1 / (1 - m)))

In [27]:
infGeomSum.relabel({x:m, m:x}).proof()

proveit.number.summation.infGeomSum

However, we are subject to **Lambda** scoping restrictions if we try to relabel in any manner such that they map to the same **Variable** (or the meaning could be changed in a manner that is not strictly weaker).

In [28]:
from proveit import ScopingViolation
try:
    infGeomSum.relabel({m:x})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Scoping violation while substitutingx -> ((Sum_{m=0}^{infinity} x^{m}) = (1 / (1 - x))) | x in Complexes.  Scoping violation while substitutingm -> x^{m} | m in {0...infinity}.  Relabeling in violation of Variable scoping restrictions.


In [29]:
try:
    infGeomSum.relabeled({x:m})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Scoping violation while substitutingx -> ((Sum_{m=0}^{infinity} x^{m}) = (1 / (1 - x))) | x in Complexes.  Relabeling in violation of Variable scoping restrictions.


In [30]:
try:
    infGeomSum.relabeled({x:n, m:n})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Scoping violation while substitutingx -> ((Sum_{m=0}^{infinity} x^{m}) = (1 / (1 - x))) | x in Complexes.  Scoping violation while substitutingm -> x^{m} | m in {0...infinity}.  Relabeling in violation of Variable scoping restrictions.


In [31]:
%end relabeling

# Next chapter: <a href="tutorial05_forall.ipynb">Universal Quantification (Forall)</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>